### Análise de sentimento

In [90]:
from textblob import TextBlob

In [94]:
frase = "Python é ótimo para Machine Learning"
tb = TextBlob(frase)
tb_en = tb.translate(from_lang='pt_br', to='en')

In [96]:
tb_en

TextBlob("Python is great for Machine Learning")

In [95]:
tb_en.sentiment.polarity

0.8

In [6]:
frase = 'Moji Mirim'
test = TextBlob(frase)

In [7]:
test.correct()

TextBlob("Foci Firm")

### Similaridade de palavras

In [17]:
# ! pip install fuzzywuzzy

In [27]:
# a lib acima esta sendo migrada para essa
# ! pip install thefuzz
# from thefuzz import fuzz

In [28]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [29]:
s1 = 'Moji Mirim'
s2 = 'Mouji Mirim'

fuzz.ratio(s1, s2)

90

In [36]:
s3 = 'Itapemirim'
fuzz.ratio(s2, s3)

40

In [37]:
s4 = 'Mougi Mirin'
fuzz.partial_ratio(s1, s4)

80

In [39]:
cidades_corretas = ['São Paulo', 'Mogi Mirim', 'Rio de Janeiro', 'Itapemirim']
cidade_class = 'Muge Merin'

### Extraindo as similaridades

In [49]:
process.extract(cidade_class, cidades_corretas, scorer=fuzz.partial_ratio)

[('Mogi Mirim', 60),
 ('Rio de Janeiro', 42),
 ('Itapemirim', 40),
 ('São Paulo', 11)]

### Limitando o Retorno

In [52]:
process.extract(cidade_class, cidades_corretas, scorer=fuzz.partial_ratio, limit=3)

[('Mogi Mirim', 60), ('Rio de Janeiro', 42), ('Itapemirim', 40)]

### Extraindo apenas o primeiro elemento com maior cutoff

In [53]:
process.extractOne(cidade_class, cidades_corretas, scorer=fuzz.partial_ratio, score_cutoff=60)

('Mogi Mirim', 60)

* Aplicar o fuzzywuzzy em uma base de dados
* Medir a similaridade de strings e fazer **Data Cleaning**

In [67]:
import pandas as pd
from collections import OrderedDict
data = OrderedDict(
    {
        'codigo_produto': [10 , 11, 12, 13, 14, 99],
        'descrição': ['iphone 6ss', 'iphone 6s', 'iphoni 6s', 'ipone 6s','Iphone 6s,,,,', 'Abacate']
        
    })

In [68]:
dataset = pd.DataFrame(data)
dataset

,codigo_produto,descrição
0,10,iphone 6ss
1,11,iphone 6s
2,12,iphoni 6s
3,13,ipone 6s
4,14,"Iphone 6s,,,,"
5,99,Abacate


In [69]:
process.extractOne('Iphone 6s', choices=dataset.descrição, scorer=fuzz.ratio, score_cutoff=95)

('iphone 6s', 100, 1)

In [70]:
def AplicaFuzzy(query, dados, metodo_ratio, score_corte):
    return process.extractOne(query, choices=dados, scorer=metodo_ratio, score_cutoff=score_corte)

In [71]:
AplicaFuzzy('Iphone 6s', dataset.descrição, fuzz.ratio, 90)

('iphone 6s', 100, 1)

In [72]:
dataset['descrição2'] = AplicaFuzzy('Iphone 6s', dataset.descrição, fuzz.ratio, 95)[0]
dataset['descrição2']

0    iphone 6s
1    iphone 6s
2    iphone 6s
3    iphone 6s
4    iphone 6s
5    iphone 6s
Name: descrição2, dtype: object

In [87]:
dataset['descrição'].apply(lambda x: AplicaFuzzy('Iphone 6s', [x], fuzz.ratio, 50))

0        (iphone 6ss, 95)
1        (iphone 6s, 100)
2         (iphoni 6s, 89)
3          (ipone 6s, 94)
4    (Iphone 6s,,,,, 100)
5                    None
Name: descrição, dtype: object

In [78]:
AplicaFuzzy('Iphone 6s', 'iphone 6ss', fuzz.ratio, 1)

('i', 20)

In [89]:
process.extract('abacate', choices=['Iphone 6s'], scorer=fuzz.ratio,)

[('Iphone 6s', 12)]